![img](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png) Filippo Miatto (2024) 

# Lecture 1: Numerical tools

# 1. A bit of numpy

`Numpy` is one of the most popular python libraries for numerical math.

The most useful tool that I want to teach you is the function `np.einsum()`, but before we get there let's learn about arrays and axes. An array is a generalization of vectors and matrices. For us an array is a collection of numbers, where each number is identified by a set of integer coordinates.

The meaning that we attribute to the array depends on what we will use it for. Here's a couple examples:

1. A complex array with shape `(n,)` can be interpreted as a vector $v \in \mathbb{C}^n$.

2. A complex array with shape `(m,n)` can be interpreted as a rectangular matrix, i.e. a map $M : \mathbb{C}^n\rightarrow \mathbb{C}^m$, but it can also be interpreted as a vector $v \in \mathbb{C}^m\otimes \mathbb{C}^n$.

A generic array is simply a collection of numbers with multiple indices: $T_{ijklmn\dots}$, whose meaning depends on the context.

The number of indices is called the _order_ of the array, so column vectors are order-1 arrays, matrices are order-2 arrays etc...

Each index has a dimension (i.e. the number of distinct integer values that it can have), and the dimension does not have to be the same for all the indices, e.g. a rectangular matrix has two indices of different dimensions. If we call $d(j)$ the dimension of index $j$, then an array $T_{j_1,\dots,j_r}$ of order $r$ contains $d(j_1)\times d(j_2)\times\dots\times d(j_r)$ numbers, and so the total size of an array scales more or less exponentially with the number of indices, i.e. with the order.

numpy provides powerful tools for handling arrays of any dimension. Here are key concepts for working with arrays:

- `shape`: A tuple containing the size of each dimension of the array. For example, a 2x3 matrix has shape (2,3). The length of the shape tuple gives the number of dimensions (order) of the array.

- `np.newaxis` (or `None`): Creates a new dimension of size 1 in an array. Useful for broadcasting and reshaping operations. Example: `v[:, None]` converts a vector into a column matrix with a single column.

- `Ellipsis` (`...`): A placeholder for any number of dimensions. Handy when you don't know the exact number of dimensions or want to keep trailing dimensions unchanged. Example: `T[..., 0]` selects the first element along the last dimension.

- `slice`: Provides efficient views into arrays without copying data. Syntax: `array[start:stop:step]`. Numpy creates a view by adjusting strides rather than copying memory.

- `Broadcasting`: Automatically aligns arrays of different shapes for operations. Rules: 1) Dimensions must match or be 1, 2) Missing dimensions are treated as 1. Example: `(3,)` array can broadcast with `(3,3)` matrix.

- `strides`: Memory layout information that determines how to traverse the array. Each dimension has a stride (bytes to skip). Understanding strides helps optimize memory access and enables advanced operations.

- `reshape`: Reorganizes array dimensions without changing data. Can split or combine dimensions. Example: `(6,)` → `(2,3)`. Use `-1` to automatically calculate a dimension size.

- `transpose`: Reorders dimensions of an array. Crucial for tensor operations and matrix manipulations. Example: `T.transpose(2,0,1)` reorders dimensions.

In [14]:
import numpy as np

# using None for new axes
v = np.array([1,2,3]) # a vector

v1 = v[:, None]
print(f'Adding a second index. New shape of the tensor is {v1.shape}:\n', v1, '\n')

v2 = v[None, :]
print(f'Adding a new first index. New shape of the tensor is {v2.shape}:\n', v2, '\n')

v3 = v[:, None, None]
print(f'Adding a second and third index. New shape of the tensor is {v3.shape}:\n', v3, '\n')

Adding a second index. New shape of the tensor is (3, 1):
 [[1]
 [2]
 [3]] 

Adding a new first index. New shape of the tensor is (1, 3):
 [[1 2 3]] 

Adding a second and third index. New shape of the tensor is (3, 1, 1):
 [[[1]]

 [[2]]

 [[3]]] 



In [15]:
# using new axes for broadcasting

v = np.array([1,2,3]) # a 3-dim vector
m = np.ones((3,3)) # a 3x3 matrix of ones

# compare:

print('broadcasting along index 1 (rows):')
print(v[None, :] * m, '\n')

print('broadcasting along index 0 (columns):')
print(v[:, None] * m)

broadcasting along index 1 (rows):
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]] 

broadcasting along index 0 (columns):
[[1. 1. 1.]
 [2. 2. 2.]
 [3. 3. 3.]]


In [16]:
# Usage of ellipsis

T = np.zeros((2,3,4,5)) # tensor of shape (2,3,4,5)
T[None, ...].shape  # adding an index of size 1

(1, 2, 3, 4, 5)

## 1.4 `np.einsum()`: the swiss army knife of array operations

`np.einsum()` (short for Einstein summation) is an incredibly powerful and versatile function in NumPy for performing a wide array of tensor operations. It can express many common operations like dot products, outer products, matrix multiplication, transpositions, traces, and sums over specific axes, as well as complex multi-dimensional contractions, all using a concise and expressive string-based notation.

The function's first argument is a string that defines the operation. This string uses letters to represent the indices of the input arrays and specifies how these indices should be combined or permuted. Subsequent arguments are the NumPy arrays themselves:

```python
np.einsum('string goes here', arrays, go, here)
```

The fundamental rule of Einstein summation is:

$$
\mathbf{Repeated\ indices\ in\ the\ input\ specification\ are\ summed\ over.}
$$

When an index is repeated (appearing in at least two input operands or twice in a single operand), it implies a summation over all possible values of that index. This process is often called "contracting" the index.

The string specification typically has two parts, separated by an arrow `->`:
-   The part **before** `->` defines the indices of the input arrays. Each array's indices are specified as a string of letters, and multiple input arrays are separated by commas (e.g., `'ij,jk'`).
-   The part **after** `->` defines the indices of the output array (e.g., `'ik'`).

If the `->` and the output indices are omitted (e.g., `'ij,jk'`), `np.einsum` infers the output: it will consist of the unrepeated input indices from the input specification, sorted alphabetically. However, explicitly defining the output using `->` is generally clearer, more robust, and recommended for precise control.

Let's explore some key examples to understand its power:

### Matrix Multiplication
Standard matrix multiplication $C = MN$ is defined as $C_{ik} = \sum_j M_{ij}N_{jk}$.
The index `j` is repeated in the input terms $M_{ij}$ and $N_{jk}$, so it is summed over (contracted). The remaining indices, `i` from $M$ and `k` from $N$, form the indices of the resulting matrix $C_{ik}$.

Matrix multiplication between, say, 4 matrices is
$$
(MNPQ)_{im} = \sum_{jkl} M_{ij}N_{jk}P_{kl}Q_{lm}
$$

With `np.einsum` this would be:

```python
np.einsum('j,jklm,jk,l -> m', M, N, P, Q)
```

### Transposition
The string after the arrow allows us to do some final rearranging of the indices, like a transposition:

$$
(MN)^T_{ki} = (MN)_{ik}
$$
With `np.einsum` this would be:

```python
np.einsum('ij,jk -> ki', M, N) # notice: ki and not ik
```

### Traces
If we repeat an index belonging to the same tensor, we compute a trace:

$$
Tr(M) = \sum_{i} M_{ii}
$$
With `np.einsum` this would be:

```python
np.einsum('ii', M)
```

### Tensor products (i.e. outer products)
Outer products are the opposite of inner products, i.e. we simply juxtapose indices:

E.g. with vectors:
$$
(\mathbf{v}\otimes \mathbf{w})_{ij} = v_{i}w_{j}
$$
With `np.einsum` this would be:

```python
np.einsum('i,j -> ij', v, w)
```

Or with matrices:
$$
(M\otimes N)_{ijkl} = M_{ij}N_{kl}
$$
With `np.einsum` this would be:

```python
np.einsum('ij,kl -> ijkl', M, N)
```

Note on the outer product; often it is a bad idea to "actually" calculate an outer product. For better performance, try to keep the parts separated unless the values of the outer product are absolutely needed. Chances are that the two parts might shed indices and get smaller by contracting with other tensors, as the calculation progresses.

### Activity 1: Hilbert-Schmidt inner product

The Hilbert-Schmidt inner product is an inner product between complex matrices and it is defined as follows:

$$
\langle M, N\rangle = Tr(M^\dagger N)
$$

where $M^\dagger$ means we transpose and complex-conjugate $M$.

- `v1`: implement the formula as is written above, using `np.matmul`, `np.trace` and `np.conj` to compute the conjugate
- `v2`: implement the formula using `np.einsum` and `np.conj`
- `v3`: implement the formula in a more efficient way than `v1` without using `np.einsum`

In [10]:
# v1
def HS_inner_product(M, N):
    return np.trace(np.matmul(np.transpose(np.conj(M)), N))

# v2
def HS_inner_product(M, N):
    return np.einsum('ji,ji', np.conj(M), N)

# v3
def HS_inner_product(M, N):
    return np.sum(np.conj(M) * N)

Notice how numpy's syntax allows us to write the inner product in a way that can apply to tensors of any (matching) shape but still look like the formula for the inner product between two vectors.


### Activity 2: Multiply by a diagonal matrix

Consider the product between three matrices: $ABC$, where $B$ is a diagonal matrix. This happens all the time when we consider eigenvalue decomposition or a singular value decomposition, where the matrix in the middle is diagonal.

- `v1`: implement this product as written above (treat $B$ as if it were any matrix) using `np.einsum`
- `v2`: implement this product in a more efficient way (use the fact that $B$ is diagonal) using `np.einsum` 
- `v3`: implement this product with `np.einsum` and `np.diag(B)`.

In [35]:
# v1
def prod_v1(A, B, C):
    return np.einsum('ij,jk,kl -> il', A, B, C) # 3 matrices
# v2
def prod_v2(A, B, C):
    return np.einsum('ij,jj,jl -> il', A, B, C) # B is diagonal
# v3
def prod_v3(A, B, C):
    return np.einsum('ij,j,jl -> il', A, np.diag(B), C) # only using the diagonal of B

In [109]:
from qernel.utils import profile


A,B,C = np.random.random((3,20,20)) # each matrix is 20 x 20
profile(prod_v1, A, B, C)
profile(prod_v2, A, B, C)
profile(prod_v3, A, B, C)


ModuleNotFoundError: No module named 'matplotlib'